<h1>Modelo Predictivo de Diabetes utilizando Machine Learning (Support Vector Machine)</h1>

<h2>Definición del Problema</h2>

<p>El objetivo de este proyecto es crear un modelo predictivo que pueda predecir si un paciente tiene diabetes o no, basado en ciertas medidas diagnósticas incluidas en el conjunto de datos. El conjunto de datos utilizado contiene información de 768 pacientes y 8 variables. El objetivo es predecir si un paciente tiene diabetes o no.</p>

<h2>Descripción SVM</h2>

<p>Support vector machine (SVM) es un algoritmo de aprendizaje supervisado que se utiliza en muchos problemas de clasificación y regresión, incluidas aplicaciones médicas de procesamiento de señales, procesamiento del lenguaje natural y reconocimiento de imágenes y voz.</p>

<p>El objetivo del algoritmo SVM es encontrar un hiperplano que separe de la mejor forma posible dos clases diferentes de puntos de datos. “De la mejor forma posible” implica el hiperplano con el margen más amplio entre las dos clases, representado por los signos más y menos en la siguiente figura. El margen se define como la anchura máxima de la región paralela al hiperplano que no tiene puntos de datos interiores. El algoritmo solo puede encontrar este hiperplano en problemas que permiten separación lineal; en la mayoría de los problemas prácticos, el algoritmo maximiza el margen flexible permitiendo un pequeño número de clasificaciones erróneas. </p>

<img src="SVM.jpeg" alt="SVM" style="width: 500px;"/>

<p>Los vectores de soporte hacen referencia a un subconjunto de las observaciones de entrenamiento que identifican la ubicación del hiperplano de separación. El algoritmo SVM estándar está formulado para problemas de clasificación binaria; los problemas multiclase normalmente se reducen a una serie de problemas binarios.</p>

<p>Específicamente, support vector machines pertenecen a una clase de algoritmos de Machine Learning denominados métodos kernel, donde se puede utilizar una función de kernel para transformar las características. Las funciones de kernel asignan los datos a un espacio dimensional diferente, que suele ser superior, con la expectativa de que resulte más fácil separar las clases después de esta transformación, simplificando potencialmente los límites de decisión complejos no lineales para hacerlos lineales en el espacio dimensional de características superior asignado. En este proceso, los datos no se tienen que transformar explícitamente, lo que supondría una alta carga computacional. Esto se conoce como truco de kernel.</p>

<p>Para este problema utilizaremos el algoritmo SVM tipo Lineal que realiza aprendizaje de dos clases:</p>

$$K(x_1,x_2)={x}^T_{1}x_{2}$$

## Importación de Librerías

In [88]:
# Librerías para trabajar y mostrar Datos
import numpy as np
import pandas as pd

# Librería para estandarizar Datos 
from sklearn.preprocessing import StandardScaler

# Librería para dividir datos en entrenamiento 
from sklearn.model_selection import train_test_split

# Maquina de vectores de soporte del modelo
from sklearn import svm

# Librería que revisa la puntuación de presición
from sklearn.metrics import accuracy_score



# Ocultar warnings
import warnings
warnings.filterwarnings('ignore')

## Recolección y Análisis de Datos

Para este proyecto utilizamos el DataSet de PIMA Diabetes Database, el cual contiene información de pacientes mujeres de al menos 21 años de edad de herencia Pima.

In [89]:
# Procedemos a cargar el CSV con los Datos con Pandas
diabetes_dataset = pd.read_csv("diabetes.csv")
diabetes_dataset

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [90]:
diabetes_dataset.shape

(768, 9)

Podemos notar que tenemos un DataSet con 9 columnas, las cuales son:

- Pregnacies: N° de Embarazos.
- Glocuse: Concentración de glucosa en plasma a las 2 horas en una prueba de tolerancia a la glucosa oral.
- BloodPressure: Presión arterial diastólica (mm Hg).
- SkinThickness: Grosor del pliegue cutáneo del tríceps (mm).
- Insulin: insulina sérica de 2 horas (mu U/ml).
- BMI: índice de masa corporal (peso en kg/(altura en m)^2).
- DiabetesPedigreeFunction: Función de pedigrí de diabetes.
- Age: Edad (años).
- Outcome: variable de clase (0 o 1).

Además de que tenemos una cantidad de 768 registros.

In [91]:
# Obtenemos la descripción estadística de los Datos.
diabetes_dataset.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [92]:
# Revisamos las Estadisticas de Diabéticos (1) y No Diabéticos (0).

print(diabetes_dataset['Outcome'].value_counts())
diabetes_dataset.groupby('Outcome').mean()

Outcome
0    500
1    268
Name: count, dtype: int64


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Outcome,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [93]:
# Separámos el DataSet en X (variables independientes) y Y (variable dependiente)

X = diabetes_dataset.drop(columns= 'Outcome',axis=1)
Y = diabetes_dataset['Outcome']

In [94]:
X,Y

(     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI   
 0              6      148             72             35        0  33.6  \
 1              1       85             66             29        0  26.6   
 2              8      183             64              0        0  23.3   
 3              1       89             66             23       94  28.1   
 4              0      137             40             35      168  43.1   
 ..           ...      ...            ...            ...      ...   ...   
 763           10      101             76             48      180  32.9   
 764            2      122             70             27        0  36.8   
 765            5      121             72             23      112  26.2   
 766            1      126             60              0        0  30.1   
 767            1       93             70             31        0  30.4   
 
      DiabetesPedigreeFunction  Age  
 0                       0.627   50  
 1                    

## Estandarización de Datos

Como podemos notar, los datos tienen diferentes escalas, por lo que es necesario estandarizarlos para que todos estén en la misma escala y así evitar que los datos con mayor escala tengan mayor peso en el modelo. Para esto utilizamos la función StandardScaler de la librería sklearn.

In [95]:
scaler = StandardScaler()

In [96]:
# Estandarizamos los 
scaler.fit(X)

StandardScaler()

In [97]:
standarized_data = scaler.transform(X)
standarized_data

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [98]:
X = standarized_data
Y = diabetes_dataset['Outcome']

## Separación de Datos de Entrenamiento y Prueba

Para entrenar el modelo utilizamos la función train_test_split de la librería sklearn, la cual nos permite dividir el DataSet en un conjunto de entrenamiento y un conjunto de prueba, en este caso utilizamos un 80% para entrenamiento y un 20% para prueba.

In [99]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y, random_state=2)


print(X.shape,X_train.shape,X_test.shape)

(768, 8) (614, 8) (154, 8)


## Entrenamiento del Modelo

Para entrenar el modelo utilizamos la función fit de la librería sklearn, la cual nos permite entrenar el modelo con los datos de entrenamiento. En este caso usaremos un modelo lineal.

In [100]:
classifier = svm.SVC(kernel='linear')

In [101]:
# Enternando el vector de soporte Machine Classifier
classifier.fit(X_train,Y_train)

SVC(kernel='linear')

## Evaluación del Modelo



In [102]:
# Realizamos la predicción del entrenemiento
X_train_predic = classifier.predict(X_train)

# Calculamos el Accurency Score
training_data_ac = accuracy_score(X_train_predic, Y_train)

print("Puntaje de Accurency del Entenamiento:", training_data_ac)

Puntaje de Accurency del Entenamiento: 0.7866449511400652


In [103]:
# Realizamos la predicción del test
X_test_predic = classifier.predict(X_test)

# Calculamos el Accurency Score
testing_data_ac = accuracy_score(X_test_predic, Y_test)

print("Puntaje de Accurency del Testing:", testing_data_ac)

Puntaje de Accurency del Testing: 0.7727272727272727


## Creando el Sistema Predictivo

In [104]:
input_data = (4,110,92,0,0,37.6,0.191,30)

# Pasamos el input a np_array
input_data_as_np_array = np.asarray(input_data)

# Reacondicionar el modelo para que use solo una instancia
input_data_reshaped = input_data_as_np_array.reshape(1,-1)

# Estandarizar los datos input
std_data = scaler.transform(input_data_reshaped)

predict = classifier.predict(std_data)

In [105]:
if (predict[0]==1):
    print("La persona podría ser Diabética")
elif (predict[0]==0):
    print("La persona puede que no sea Diabética")
else:
    print("Error en la predicción")

La persona puede que no sea Diabética
